In [1]:
import openai
import os
import json
from dataclasses import dataclass
from typing import List
# openai.api_key = os.environ.get("OPEN_AI_FREE_API_KEY")
openai.api_base = 'https://api.pawan.krd/v1'
openai.api_key = "pk-gJLIzltadzOmCjcGgkvKCLALZbXxBwwJkeyGuJcxYSSdWZDK"


In [2]:
from prompts import IN, OUT, NEW_CHAR_SYS_PROMPT, NEW_CHAR_SYS_PROMPT_PLUS, tcol

# IN = "⁂⁂⁂⁂"
# OUT = "===="
# vdots = "⋮"
# dots = "…"
# tcol = "⁝"
# asterism = "⁂"
# bullet = "•"
# quadprime = "⁕"
# bbar = "¦"
# dbar = "‖"
# ascii_fun_chars = "⩶•—–◊▲△◁▷▼▽◤◣◢◥∀∃¦|⩨⫴⫲⫵⁅⁆⟪⟫⟦⟧‖‖⁕⁂✓✕⟨⟩"


In [3]:
class Machine:
    def __init__(self, sys_prompt, model="gpt-3.5-turbo", temperature = 0):
        self._sys_messages = [{"role": "system", "content": sys_prompt}]
        self.response = ""
        self._model = model
        self._temperature = temperature
        
    def set_system(self, sys_prompt):
        self._sys_messages = [{"role": "system", "content": sys_prompt}]
    
    def add_system(self, sys_prompt):
        self._sys_messages.append({"role": "system", "content": sys_prompt})

    def get_completion(self, prompt, role = "user"):
        messages = self._sys_messages + [{"role": role, "content": prompt}]
        response = openai.ChatCompletion.create(
            model=self._model,
            messages=messages,
            temperature=self._temperature, # this is the degree of randomness of the model's output
            max_tokens=500,
        )
        self.response = response.choices[0].message["content"]
        return self.response


In [11]:
from prompts import IN, OUT, NEW_CHAR_SYS_PROMPT, NEW_CHAR_SYS_PROMPT_IMPROVED, NEW_CHAR_SYS_PROMPT_PLUS, tcol

class CreateChar(Machine):
    
    sys_prompt = NEW_CHAR_SYS_PROMPT_PLUS

    def __init__(self, character_initial_json):
        super().__init__(CreateChar.sys_prompt)
        start_char_prompt = f"""
                {IN}
                {character_initial_json}
                {IN}"""

        self.response = self.get_completion(start_char_prompt)
        json_error = True
        count = 0
        # while json_error:
        while count < 5 and json_error:
                try:
                       self.json_response = json.loads(self.response)
                except Exception as e:
                        print(e)
                        print(self.response)
                else:
                        json_error = False
                finally:
                        if json_error:
                                self.response = self.get_completion(start_char_prompt)
                count += 1     
                
        self.sheet_dict = dict(self.json_response)

# "action": <a string with the next action the character will perform. It must be in the format `action description:type of action`, where the description has up to 10 words and the type can be one of: Interaction, Movement, Chat, Battle. Must not be empty.>,\


class Agent(Machine):
    char_sys_prompt = lambda sheet_dict: f"""
        You are an autonomous agent character. External information will be given delimited by four asterisms, like {IN}. Outputs will be delimited by four equal signs, like in {OUT}.
        Metadata, metainformation, metavariables or meta-instructions will be given delimited by angled brackets, like <some instruction here>. 
        You should interpret such data, information, variables and instructions by appropriately considering the context wherein they appear.
        When needed, you will be provided with extra information to help you understand the context. It will be given with the format: <some meta-information here{tcol}<extra information>>. 
        Your current state is given by the data in the JSON object found within the following delimiters:
        {IN}
        {sheet_dict}
        {IN}
        
        You will be given a user prompt with the following possible formats:
        {IN} @perceive {IN} in which case you provide a description of the current state of the world at your surroundings, based on the description of the location and on your state data. Make sure to list all perceived objects and characters, and all relations between them, and inferences about them.
        {IN} @act {IN} in which case you provide a description of next single action, based on all the information available about your current state, about your surroundings and about possible other agents and/or objects in the vicinities.
        You must provide the next action you take in the following format:
        {OUT}
        "action": <a string with the next action the character will perform. It must be in the format `action description:type of action`, where the description has up to 20 words and the type can be one of: Interaction, Movement, Chat, Battle. Must not be empty.>,
        {OUT}
        """
    def __init__(self, sheet_dict):
        super().__init__(Agent.char_sys_prompt(sheet_dict))
        self.sheet_dict = sheet_dict
        for key, value in self.sheet_dict.items():
            setattr(self, key, value)
    def __str__(self):
        return f"{self.name}: {self.description}"
    
    def perceive(self):
        perceive_prompt = f"""
            {IN} @perceive {IN}
        """
        self.perceived = self.get_completion(perceive_prompt)
        # self.memories.append(self.response)
        return self.perceived
    
    def act(self):
        perceive_prompt = f"""
            {IN} @act {IN}
        """
        self.action = self.get_completion(perceive_prompt)
        # self.memories.append(self.response)
        return self.action



In [12]:
name = "John"
character_concept = "A traveling bard who seeks to spread joy and laughter through his music."
starting_location = 'The Arena'
starting_location_description = 'The Arena is a small and circular location with a beautiful marble floor and a magnificent dome on top. The walls of the arena are made of sandstone and they are adorned with intricate engravings that depict ancient battles and warriors. The dome is high and adorned with golden patterns that reflect the light beautifully around the arena. The place can accommodate a small audience sitting around, providing a great view of the arena and its center. A beautiful mural can be found on one of the walls, depicting the god of war in a fierce pose.'
starting_location_coordinates = '(7:24, 3:17)'

john_initial_dict = {"name": "John",
"character_concept": "A traveling bard who seeks to spread joy and laughter through his music.",
"starting_location": 'The Arena',
"starting_location_description": 'The Arena is a small and circular location with a beautiful marble floor and a magnificent dome on top. The walls of the arena are made of sandstone and they are adorned with intricate engravings that depict ancient battles and warriors. The dome is high and adorned with golden patterns that reflect the light beautifully around the arena. The place can accommodate a small audience sitting around, providing a great view of the arena and its center. A beautiful mural can be found on one of the walls, depicting the god of war in a fierce pose.',
"starting_location_coordinates": '(7:24, 3:17)'}

john_initial_json = json.dumps(john_initial_dict)

In [13]:
create_john = CreateChar(john_initial_json)
print(create_john.json_response)
john_sheet_dict = create_john.sheet_dict

{'name': 'John', 'level': 2, 'health': 7, 'concept': 'A traveling bard who seeks to spread joy and laughter through his music.', 'location': 'The Arena', 'appearance': 'Young, flamboyant bard with a colorful outfit and a lute.', 'personality': 'Charismatic, outgoing, and empathetic.', 'abilities': 'Skilled in music and storytelling, able to control crowds.', 'backstory': 'John grew up in a poor village and discovered his passion for music at an early age. He left home to become a bard, spreading joy and laughter wherever he goes.', 'memories': ['The joy on the faces of his first audience', "The sound of his mother's voice singing to him at night", 'The smell of the spring flowers in his village'], 'goals': ['Minute⁝Entertain the audience with a new song', 'Hour⁝Find a wealthy patron to sponsor his travels', 'Week⁝Discover a lost song that can win the hearts of even the toughest audience'], 'plans': ['Minute⁝Start a lively tune and encourage audience participation', 'Hour⁝Find a noble w

In [14]:
john= Agent(john_sheet_dict)

In [15]:
json.loads(create_john.response)

{'name': 'John',
 'level': 2,
 'health': 7,
 'concept': 'A traveling bard who seeks to spread joy and laughter through his music.',
 'location': 'The Arena',
 'appearance': 'Young, flamboyant bard with a colorful outfit and a lute.',
 'personality': 'Charismatic, outgoing, and empathetic.',
 'abilities': 'Skilled in music and storytelling, able to control crowds.',
 'backstory': 'John grew up in a poor village and discovered his passion for music at an early age. He left home to become a bard, spreading joy and laughter wherever he goes.',
 'memories': ['The joy on the faces of his first audience',
  "The sound of his mother's voice singing to him at night",
  'The smell of the spring flowers in his village'],
 'goals': ['Minute⁝Entertain the audience with a new song',
  'Hour⁝Find a wealthy patron to sponsor his travels',
  'Week⁝Discover a lost song that can win the hearts of even the toughest audience'],
 'plans': ['Minute⁝Start a lively tune and encourage audience participation',
 

In [16]:
john.perceive()

"I look around at my surroundings and perceive that I'm currently in The Arena, a place where people gather to watch performances. The walls are made of sandstone and the white marble floor contrasts with it sharply. I notice a high dome above which reflects the light, illuminating a central stage where I'm going to perform soon. The bleachers are crowded with expectant people, eagerly waiting for my show."

In [17]:
john.act()

'Based on my current state and surroundings, I think the best course of action would be to begin my performance. I will aim to entertain the crowd, increase my reputation, and hopefully attract a wealthy patron to sponsor my travels. \n\n====\n"action": "Start playing an energetic tune to attract the audience and encourage participation. :Interaction",'

In [18]:
name = "Maskaz"
character_concept = "A maverick mathematician who is bored and seeks to fulfill he's existential anxiety with senseless adventures and pointless gunfires."
starting_location = 'The Arena'
starting_location_description = 'The Arena is a small and circular location with a beautiful marble floor and a magnificent dome on top. The walls of the arena are made of sandstone and they are adorned with intricate engravings that depict ancient battles and warriors. The dome is high and adorned with golden patterns that reflect the light beautifully around the arena. The place can accommodate a small audience sitting around, providing a great view of the arena and its center. A beautiful mural can be found on one of the walls, depicting the god of war in a fierce pose.'
starting_location_coordinates = '(7:24, 3:17)'

maskaz_initial_dict = {"name": name,
"character_concept": character_concept,
"starting_location": starting_location,
"starting_location_description": starting_location_description,
"starting_location_coordinates": starting_location_coordinates}

maskaz_initial_json = json.dumps(maskaz_initial_dict)

In [19]:
maskaz_initial_json, john_initial_json

('{"name": "Maskaz", "character_concept": "A maverick mathematician who is bored and seeks to fulfill he\'s existential anxiety with senseless adventures and pointless gunfires.", "starting_location": "The Arena", "starting_location_description": "The Arena is a small and circular location with a beautiful marble floor and a magnificent dome on top. The walls of the arena are made of sandstone and they are adorned with intricate engravings that depict ancient battles and warriors. The dome is high and adorned with golden patterns that reflect the light beautifully around the arena. The place can accommodate a small audience sitting around, providing a great view of the arena and its center. A beautiful mural can be found on one of the walls, depicting the god of war in a fierce pose.", "starting_location_coordinates": "(7:24, 3:17)"}',
 '{"name": "John", "character_concept": "A traveling bard who seeks to spread joy and laughter through his music.", "starting_location": "The Arena", "s

In [20]:
create_maskaz = CreateChar(maskaz_initial_json)
print(create_maskaz.json_response)
maskaz_sheet_dict = create_maskaz.sheet_dict

Unterminated string starting at: line 33 column 21 (char 1783)
{
    "name": "Maskaz",
    "level": 2,
    "health": 10,
    "concept": "A maverick mathematician seeking senseless adventures",
    "location": "The Arena",
    "appearance": "A lanky man with disheveled dark hair and a pair of glasses",
    "personality": "Charismatic, adventurous and impulsive at times",
    "abilities": "Expert in probability theory, excellent marksman with firearms",
    "backstory": "Maskaz was a brilliant mathematician who grew bored of the academic world. Seeking a sense of purpose, he turned to the life of adventuring, feeling that he would be able to find meaning in a world of danger and excitement. He has since traveled far and wide, taking on odd jobs and attending different arenas and gun shows.",
    "memories": [
        "Winning his first gun competition by a hair's breadth",
        "A near-death experience with a hired gun in his last adventure",
        "The joy he felt when discovering 

In [23]:
print(create_maskaz.response)

{
    "name": "Maskaz",
    "level": 2,
    "health": 8,
    "concept": "A maverick mathematician seeking senseless adventures.",
    "location": "The Arena",
    "appearance": "A tall, lanky fellow with thick-framed glasses.",
    "personality": "Maskaz is immensely curious and always willing to take risks.",
    "abilities": "Maskaz is highly skilled in probability and statistics.",
    "backstory": "Maskaz spent months in seclusion, scribbling on a notebook and analyzing data. After a while, he realized that there might be more to life than math. By sheer luck, he stumbled upon the Arena and decided to join.",
    "memories": [
        "Lived alone for a long period of time.",
        "Never left his city before.",
        "Has a knack for puzzles."
    ],
    "goals": [
        "Minute⁝To win the next round.",
        "Hour⁝To gather some intel about other fighters.",
        "Week⁝To become the champion of the Arena."
    ],
    "plans": [
        "Minute⁝Observe how the other fig

In [24]:
maskaz = Agent(maskaz_sheet_dict)

In [25]:
maskaz.perceive()

"Maskaz looks around him and sees the massive stone wall that surrounds the Arena with seven entrances marked by numbers. The crowd is clamoring and the heat is sweltering. He notices a few fighters already on the arena, some stretching and others practicing their moves. He also sees the bookkeeper's booth where he might be able to gather some intel."

In [26]:
maskaz.act()

'I observe the fighters around me, looking for any patterns or weaknesses in their movements. As I look around, I notice someone who seems to be deliberately avoiding eye contact and moving around surreptitiously. This person raises my interest and I decide to approach them to gather some intel. \n\n ====\n        "action": "Approach suspicious fighter:Interaction",\n        ===='

In [32]:
john.perceive()

'As John looks around The Arena, he sees a few people watching a group of fighters who are currently practicing. The ground is covered in sand and there is a ring in the center where contests and performances take place. He notices some weapons and shields scattered around the edges of the arena. From his vantage point, John cannot see any entrances or exits to The Arena.'

In [33]:
john.act()

'After observing the fighters practicing in the arena, John feels inspired to perform and spread joy through his music. He plans to approach the group and offer to play a song or two for them. \n\n"action": "Approach the group of fighters and ask if they would like to listen to some music: Interaction"'